# Установка пакетов, догрузка необходимых файлов и функций

In [ ]:
!pip install pyteomics
!pip install biopython
!pip install modlamp
!pip install propy3 
!pip install PyPro

     |████████████████████████████████| 224 kB 8.6 MB/s 
     |████████████████████████████████| 2.3 MB 7.4 MB/s 
     |████████████████████████████████| 8.0 MB 7.5 MB/s 
     |████████████████████████████████| 154 kB 51.5 MB/s 
     |████████████████████████████████| 13.1 MB 53.6 MB/s 
  Created wheel for modlamp: filename=modlamp-4.3.0-py3-none-any.whl size=173903 sha256=579da1eb79a348d36372158dde92bc6268b49a34bb2d0e3891070ebe00e7b759
  Stored in directory: /root/.cache/pip/wheels/89/2a/d3/d878765a345146eebefa0fd9feb33ba35b6bdf074530f4fb67
Successfully built modlamp
     |████████████████████████████████| 278 kB 6.8 MB/s 
  Created wheel for PyPro: filename=pypro-0.1.0-py3-none-any.whl size=6685 sha256=6bed56cb2300b2d8b43e2fb9f48234eb310ca909f09b96dda71e530dc03eada3
  Stored in directory: /root/.cache/pip/wheels/18/20/31/f662e327173157125ad5dbae04d4b85eaccf3c2e13b99c55c9
Successfully built PyPro


In [ ]:
import csv
import joblib
import pandas as pd
import pyteomics.parser as parser
import re
import time
import xgboost
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis 
from propy import CTD
from propy import PyPro
from pyteomics.parser import cleave, expasy_rules
from sklearn import svm
from sklearn.metrics import mean_squared_error, make_scorer, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from modlamp.database import query_database
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
from modlamp.sequences import Helices

In [ ]:
#Пути к файлам
Total_models_path = '2. Model training and parameter estimation/Total models/'
Organisms_path = '3. Search for new CPP/Organisms/'
Results_path = '3. Search for new CPP/Results/'

#Часто используемые переменные
CPP_test_file = '1. Sample creation/Total/df_SAMPLE.csv'
animals = ['Spider', 'Jellyfish', 'Red ant', 'Honeybee', 'Snake', 'Leech']
proteome = [True, True, False, False, False, False]

#Модели
KNN_model = joblib.load(Total_models_path + 'KN_model_final.pkl')
XGB_model = joblib.load(Total_models_path + 'XGB_model_final.pkl')
RFM_model = joblib.load(Total_models_path + 'RF_model_final.pkl')

#Организмы
spider_file = 'SPIDER.fasta'
jellyfish_file = 'JELLYFISH.xlsx'
red_ant_file = 'RED_ANT.xlsx'
honeybee_file = 'HONEYBEE.xlsx'
snake_file = 'SNAKE.xlsx'
leech_file = 'LEECH.fasta'
animals_file = [spider_file, jellyfish_file, red_ant_file, 
                honeybee_file,snake_file, leech_file]

spider_result = 'spider.csv'
jellyfish_result = 'jellyfish.csv'
red_ant_result = 'red_ant.csv'
honeybee_result = 'honeybee.csv'
snake_result = 'snake.csv'
leech_result = 'leech.csv'
animals_result = [spider_result, jellyfish_result, red_ant_result, 
                  honeybee_result, snake_result, leech_result]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or inv

In [ ]:
#Файл ---> Список последовательностей
def reading_sequences_from_file(file_name):
    file_format = file_name.split('.')[-1]
    file_name = Organisms_path + file_name
    if file_format == 'xlsx':
        animal = pd.read_excel(file_name)['Peptide'].drop_duplicates()
    if file_format == 'csv':
        animal = pd.read_csv(file_name)['Peptide'].drop_duplicates()
    if file_format == 'fasta':
        animal = list()
        with open(file_name) as file:
            seqs = SeqIO.parse(file, "fasta")
            for seq in seqs:
                animal.append(str(seq.seq))
        animal = list(set(animal))
    return animal

#Список белков ---> Словарь белков и пептидов + Список пептидов
def proteins_for_peptides(animal_proteins):
    result = dict()
    peptides = list()
    for protein in animal_proteins:
        result[protein] = []
        for rule in parser.expasy_rules.keys():
            pep = cleave(protein, expasy_rules[rule])
            for p in pep:
                result[protein].append(p)
                peptides.append(p)
    return result, list(set(peptides))

#Пептиды ---> Очищенные от лишних символов пептиды заданной длины с заданным ак составом, ранее не известные как ПП
def purify_peptides(peptides, min_len, max_len, max_per, test_file_name):
    test_file = pd.read_csv(test_file_name)
    clean_peptides = list()
    for pep in peptides:
        pep = re.sub(r'[^NDCQEGHIMFSTWYPKARLV]', '', pep)
        if min_len <= len(pep) <= max_len:
            if pep in test_file['Sequence']:
                print('Used in training')
            else:
                start_dict = ProteinAnalysis(pep).count_amino_acids()
                per = max(start_dict.values()) / (len(pep) * 0.01)
                if per <= max_per:
                    clean_peptides.append(pep)
    return list(set(clean_peptides))

#Вспомогательный шаг для записи дескрипторов в словарь
def create_a_dictionary(feature, parameter):
    if len(feature) == 0:
        for key in parameter.keys():
            feature[key] = [parameter[key]]
    else:
        if parameter != 0:
            for key in parameter.keys():
                feature[key].append(parameter[key])
        else:
            for key in feature.keys():
                feature[key].append(0)
    return feature

#Список отфильтрованных пептидов ---> Дескрипторы
def count_descriptors(peptides):
    desc_names = ['_ChargeD1025', '_ChargeD1075', '_ChargeD1100', 
                  '_ChargeD3001', '_ChargeD3100',
                  '_NormalizedVDWVD1001', '_NormalizedVDWVD2001',
                  '_NormalizedVDWVD2025', '_NormalizedVDWVD2075',
                  '_PolarityD1075', '_PolarityD3025',
                  '_PolarizabilityD1100',
                  '_SecondaryStrD2025', '_SecondaryStrD2075', '_SecondaryStrD2100',
                  '_SecondaryStrD3025', '_SecondaryStrD3100',
                  '_SolventAccessibilityD3025', '_SolventAccessibilityD3100']
    CTD = {}
    for pep in peptides:
        DesObject = PyPro.GetProDes(pep)
        try:
            ctd = DesObject.GetCTD()
        except ZeroDivisionError:
            ctd = 0
        CTD = create_a_dictionary(CTD, ctd)
    if CTD != {}:
        df_desc = pd.DataFrame(CTD)[desc_names].copy()
    else:
        print('Something went wrong')
        return CTD
    AMP = GlobalDescriptor(peptides)
    AMP.instability_index()
    i_index = pd.DataFrame({"InstabilityInd": AMP.descriptor.flatten().tolist()})
    df_desc = pd.concat([df_desc, i_index], axis = 1)
    return df_desc

#Список пептидов с дескрипторами ---> Список пептидов с результатом работы алгоритмов ---> Запись результатов в файл
def launch_ML_search(peptides, descriptors, name_to_save):
    scaler = StandardScaler()
    scal_desc = scaler.fit_transform(descriptors)
    df_peptides = pd.DataFrame(peptides).rename(columns={0: 'Peptide'})
    df_knn = pd.DataFrame(KNN_model.predict_proba(scal_desc)).round(3)
    df_xgb = pd.DataFrame(XGB_model.predict_proba(scal_desc)).round(3)
    df_rfm = pd.DataFrame(RFM_model.predict_proba(scal_desc)).round(3)
    df_animal = pd.concat([df_peptides, df_knn], axis = 1).rename(columns={1: 'CPP_KNN', 0: 'nonCPP_KNN'})
    df_animal = pd.concat([df_animal, df_xgb], axis = 1).rename(columns={1: 'CPP_XGB', 0: 'nonCPP_XGB'})
    df_animal = pd.concat([df_animal, df_rfm], axis = 1).rename(columns={1: 'CPP_RFM', 0: 'nonCPP_RFM'})
    name_to_save = Results_path + name_to_save
    #df_animal[(df_animal.CPP_XGB >= 0.5) & (df_animal.CPP_KNN >= 0.5) & (df_animal.CPP_RFM >= 0.5)].to_csv(name_to_save)
    df_animal.to_csv(name_to_save)
    return df_animal

#Список пептидов с результатом работы алгоритмов ---> Список кандидатных CPP пептидов
def predict_CPP(df_animal, xgb_limit, knn_limit, rfm_limit):
    start_time = time.time()
    print('KNN model \t' + str((df_animal['CPP_KNN'] > knn_limit).sum()))
    print('XGB model \t' + str((df_animal['CPP_XGB'] > xgb_limit).sum()))
    print('RF model \t' + str((df_animal['CPP_RFM'] > rfm_limit).sum()))
    df_animal = df_animal[(df_animal.CPP_XGB >= xgb_limit) & (df_animal.CPP_KNN >= knn_limit) & (df_animal.CPP_RFM >= rfm_limit)]
    print('KNN + XGB + RFM \t' + str(len(df_animal)))
    #predicted_CPP = df_animal['Peptide']
    print('Проникающие пептиды предсказаны за' + time_to_do(start_time, time.time()))
    return df_animal

#Список пептидов с результатом работы алгоритмов ---> Список кандидатных nonCPP пептидов
def predict_nonCPP(df_animal, xgb_limit, knn_limit, rfm_limit):
    start_time = time.time()
    print('KNN model \t' + str((df_animal['nonCPP_KNN'] > knn_limit).sum()))
    print('XGB model \t' + str((df_animal['nonCPP_XGB'] > xgb_limit).sum()))
    print('RF model \t' + str((df_animal['nonCPP_RFM'] > rfm_limit).sum()))
    df_animal = df_animal[(df_animal.nonCPP_XGB >= xgb_limit) & (df_animal.nonCPP_KNN >= knn_limit) & (df_animal.nonCPP_RFM >= rfm_limit)]
    print('KNN + XGB + RFM \t' + str(len(df_animal)))
    #predicted_CPP = df_animal['Peptide']
    print('Непроникающие пептиды предсказаны за' + time_to_do(start_time, time.time()))
    return df_animal

#Время выполнения функции
def time_to_do(start, end):
    t = end - start
    total = '\t' + str(int(t // 60)) + ' min ' + str(round(t % 60)) + ' sec'
    return total

#Запуск одной функцией
def CPP_search(file_name, flag, min_len, max_len, max_per, CPP_test_file, name_to_save):
    start_time = time.time()
    animal_sequences = reading_sequences_from_file(file_name)
    print('Файл прочитан за' + time_to_do(start_time, time.time()))

    if flag:
        start_time = time.time()
        animal_dict, animal_peptides = proteins_for_peptides(animal_sequences)
        print('Белки разделены на пептиды за' + time_to_do(start_time, time.time()))
    else:
      animal_peptides = animal_sequences

    start_time = time.time()
    clean_animal_peptides = purify_peptides(animal_peptides, min_len, max_len, max_per, CPP_test_file)
    print('Пептиды очищены за' + time_to_do(start_time, time.time()))

    start_time = time.time()
    df_animal_desc = count_descriptors(clean_animal_peptides)
    print('Дескрипторы посчитаны за' + time_to_do(start_time, time.time()))

    start_time = time.time()
    df_animal = launch_ML_search(clean_animal_peptides, df_animal_desc, name_to_save)
    print('Результаты работы алгоритмов получены за' + time_to_do(start_time, time.time()))

    return df_animal

# Поиск проникающих пептидов

In [ ]:
min_len = 9     #Минимальная длина пептида
max_len = 35    #Максимальная длина пептида
max_per = 30    #Процент уникальности аминокислот
df_animals = dict()
organisms = zip(animals, animals_file, animals_result, proteome)
for animal, animal_file, animal_result, prot in organisms:
    print('\n\t', animal)
    df_an = CPP_search(animal_file, prot, min_len, max_len, max_per, CPP_test_file, animal_result)
    df_animals[animal] = df_an


	 Spider
Файл прочитан за	0 min 1 sec
Белки разделены на пептиды за	0 min 54 sec
Пептиды очищены за	1 min 17 sec
Дескрипторы посчитаны за	11 min 8 sec
Результаты работы алгоритмов получены за за	3 min 5 sec

	 Jellyfish
Файл прочитан за	0 min 0 sec
Белки разделены на пептиды за	0 min 21 sec
Пептиды очищены за	1 min 21 sec
Дескрипторы посчитаны за	11 min 48 sec
Результаты работы алгоритмов получены за за	3 min 11 sec

	 Red ant
Файл прочитан за	0 min 1 sec
Пептиды очищены за	0 min 1 sec
Дескрипторы посчитаны за	0 min 1 sec
Результаты работы алгоритмов получены за за	0 min 1 sec

	 Honeybee
Файл прочитан за	0 min 2 sec
Пептиды очищены за	0 min 0 sec
Дескрипторы посчитаны за	0 min 0 sec
Результаты работы алгоритмов получены за за	0 min 0 sec

	 Snake
Файл прочитан за	0 min 0 sec
Пептиды очищены за	0 min 0 sec
Дескрипторы посчитаны за	0 min 0 sec
Результаты работы алгоритмов получены за за	0 min 0 sec

	 leech
Файл прочитан за	0 min 0 sec
Пептиды очищены за	0 min 0 sec
Дескрипторы посчита

## Паук-мегаломорф Hadronyche infensa

In [ ]:
xgb_limit = 0.95
knn_limit = 0.95
rfm_limit = 0.95
predict_CPP(df_animals['Spider'], xgb_limit, knn_limit, rfm_limit)

XGB model 	80357
KNN model 	30444
RF model 	12
XGB + KNN + RFM 	11
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
59403,TSYNAALVNGGGVKMPDR,0.004,0.996,0.0,1.0,0.047,0.953
252989,QPGCVWILTSTASMGHKAMSAKNSAE,0.010,0.990,0.0,1.0,0.047,0.953
289068,NSPICPALSGKGAPDPDAEDAR,0.006,0.994,0.0,1.0,0.044,0.956
399730,WSPQIGSMLTNSYRPLAEHGR,0.002,0.998,0.0,1.0,0.047,0.953
400888,SQLSTLGNLGGSPKDQADR,0.006,0.994,0.0,1.0,0.047,0.953
415797,SHCRPPHGAEGHR,0.001,0.999,0.0,1.0,0.044,0.956
523602,CHSVSMGLTNQKVHSDR,0.002,0.998,0.0,1.0,0.041,0.959
537441,PQICSINTDAMKSRL,0.004,0.996,0.0,1.0,0.044,0.956
576375,PQPRVTGLAESAGK,0.005,0.995,0.0,1.0,0.047,0.953
824290,SWQGSAVVSIDSKRARAAS,0.003,0.997,0.0,1.0,0.037,0.963


In [ ]:
xgb_limit = 0.0
knn_limit = 0.0
rfm_limit = 0.0
pred = predict_CPP(df_animals['Spider'], xgb_limit, knn_limit, rfm_limit)
pred[pred['Peptide'] == 'CRYFHYRQKKHWQL']

KNN model 	1075083
XGB model 	1119698
RF model 	1119699
KNN + XGB + RFM 	1119699
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
843269,CRYFHYRQKKHWQL,0.469,0.531,0.222,0.778,0.39,0.61


## Медузы ропилема Rhopilema esculentum и амакуза Sanderia malayensis

In [ ]:
xgb_limit = 0.95
knn_limit = 0.95
rfm_limit = 0.95
predict_CPP(df_animals['Jellyfish'], xgb_limit, knn_limit, rfm_limit)

KNN model 	31136
XGB model 	81667
RF model 	6
KNN + XGB + RFM 	5
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
69067,SYQWQIFYRSLDGSGAKE,0.005,0.995,0.0,1.0,0.041,0.959
422503,CQNTQVNISNQHRPAKMDGK,0.005,0.995,0.0,1.0,0.041,0.959
633258,SHSLGKAPDGSGR,0.002,0.998,0.0,1.0,0.034,0.966
643661,TVANFKTNSAAPPAAEPPR,0.003,0.997,0.0,1.0,0.044,0.956
716405,TNHNYVRFHHSHHQQDDGDGK,0.004,0.996,0.0,1.0,0.034,0.966


## Красный муравей Manica rubida

In [ ]:
xgb_limit = 0.95
knn_limit = 0.95
rfm_limit = 0.85
predict_CPP(df_animals['Red ant'], xgb_limit, knn_limit, rfm_limit)

XGB model 	197
KNN model 	45
RF model 	4
XGB + KNN + RFM 	4
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
1342,HTGSLLAVVSLLLLKLVAPAAAEVLAGGKLP,0.004,0.996,0.0,1.0,0.112,0.888
1481,PTPAQKALTMLTMLLALLPVPACLEAEKYG,0.010,0.990,0.0,1.0,0.108,0.892
2546,PKGVTGAAAAPVVKLLKAAVAPDPLGKAPQ,0.002,0.998,0.0,1.0,0.146,0.854
2643,PQAVLFVLLKLLLKVAPAAAEVKGHS,0.002,0.998,0.0,1.0,0.051,0.949


In [ ]:
xgb_limit = 0.0
knn_limit = 0.0
rfm_limit = 0.0
pred = predict_CPP(df_animals['Red ant'], xgb_limit, knn_limit, rfm_limit)
#pred[pred['Peptide'] == 'KHLKHTPVWWY']
pred[pred['Peptide'] == 'RGSLLAKAALKRS']

KNN model 	2657
XGB model 	2760
RF model 	2760
KNN + XGB + RFM 	2760
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
1326,RGSLLAKAALKRS,0.609,0.391,0.333,0.667,0.471,0.529


## Медоносная пчела Apis mellifera

In [ ]:
xgb_limit = 0.95
knn_limit = 0.95
rfm_limit = 0.70
predict_CPP(df_animals['Honeybee'], xgb_limit, knn_limit, rfm_limit)

XGB model 	32
KNN model 	6
RF model 	17
XGB + KNN + RFM 	2
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
4,CLHYTVDKSKPK,0.007,0.993,0.0,1.0,0.203,0.797
358,SVCPPQLLVFDLNTSQLLK,0.002,0.998,0.0,1.0,0.224,0.776


## Королевская кобра Ophiophagus hannah

In [ ]:
xgb_limit = 0.90
knn_limit = 0.90
rfm_limit = 0.70
predict_CPP(df_animals['Snake'], xgb_limit, knn_limit, rfm_limit)

KNN model 	14
XGB model 	19
RF model 	8
KNN + XGB + RFM 	2
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
35,RAVTIFGESAGAASVGMHLLSTQSRA,0.015,0.985,0.056,0.944,0.20,0.80
248,KTWHMVYPGGYDHTRG,0.002,0.998,0.000,1.000,0.19,0.81


In [ ]:
xgb_limit = 0.0
knn_limit = 0.0
rfm_limit = 0.0
pred = predict_CPP(df_animals['Snake'], xgb_limit, knn_limit, rfm_limit)
pred[pred['Peptide'] == 'REKDLLPRK']

KNN model 	238
XGB model 	252
RF model 	252
KNN + XGB + RFM 	252
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
115,REKDLLPRK,0.972,0.028,0.944,0.056,0.671,0.329


## Медицинская пиявка Hirudo medicinalis

In [ ]:
xgb_limit = 0.95
knn_limit = 0.87
rfm_limit = 0.70
predict_CPP(df_animals['leech'], xgb_limit, knn_limit, rfm_limit)

XGB model 	24
KNN model 	17
RF model 	14
XGB + KNN + RFM 	3
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
35,AIRNDEELNK,0.023,0.977,0.111,0.889,0.200,0.800
140,SKAADESER,0.008,0.992,0.056,0.944,0.180,0.820
264,ALVVDNGSGMCKAGFAGDDAPR,0.007,0.993,0.111,0.889,0.183,0.817


In [ ]:
#nonCPP
xgb_limit = 0.95
knn_limit = 0.95
rfm_limit = 0.65
predict_nonCPP(df_animals['leech'], xgb_limit, knn_limit, rfm_limit)

XGB model 	18
KNN model 	9
RF model 	54
XGB + KNN + RFM 	2
Непроникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
9,DLLSGVLGGVDN,0.960,0.040,1.0,0.0,0.715,0.285
27,DLLSGVLGGVDDLASLDVAG,0.986,0.014,1.0,0.0,0.698,0.302


In [ ]:
xgb_limit = 0.0
knn_limit = 0.0
rfm_limit = 0.0
pred = predict_CPP(df_animals['leech'], xgb_limit, knn_limit, rfm_limit)
pred[pred['Peptide'] == 'HYNKRSTIT']

KNN model 	303
XGB model 	312
RF model 	312
KNN + XGB + RFM 	312
Проникающие пептиды предсказаны за	0 min 0 sec


,Peptide,nonCPP_XGB,CPP_XGB,nonCPP_KNN,CPP_KNN,nonCPP_RFM,CPP_RFM
94,HYNKRSTIT,0.951,0.049,1.0,0.0,0.644,0.356
